In [1]:
%load_ext autoreload

In [2]:
%autoreload
import numpy as np
import scipy.special as sp
from mpmath import exp as mp_exp
from mpmath import log as mp_log
from mpmath import polylog as mp_polylog
from scipy.integrate import quad
import darkhistory.physics as phys
import darkhistory.utilities as utils

np.set_printoptions(precision=20)

In [3]:
sp.bernoulli(5)

array([ 1.                    , -0.5                   ,
        0.16666666666666665741,  0.                    ,
       -0.03333333333327591352,  0.                    ])

In [4]:
def F1(a, b):
    def indef_int(x):
        if x < 0.01:
            # Left off -pi**2/6 so that we are only adding small numbers.
            return x - x**2/4 + x**3/36 - x**5/3600 + x**7/211680 - x**9/10886400
        else:
            return x*np.log(1. - np.exp(-x)) - sp.spence(np.array(1. - np.exp(-x), dtype='float64'))  + np.pi**2/6
#     print(-np.pi**2/6+indef_int(b))
#     print(-np.pi**2/6+indef_int(a))
    return indef_int(b) - indef_int(a)

def F0(a,b):
    def indef_int(x):
        if x < 0.001:
            return np.log(x) - x/2 + x**2/24 - x**4/2880 + x**6/181440 - x**8/9676800 + x**10/479001600
        return np.log(1. - np.exp(-x))
    return indef_int(b) - indef_int(a)

In [5]:
print(13.51975*np.log(1.-np.exp(-13.51975))-sp.spence(1-np.exp(-13.51975)))

-1.95167095473e-05


In [6]:
def H(a, b, tol=1e-30):
    """ a and b are now vectors, corresponding to a list of lower and upper bounds."""
    
    bound = 2
    
    def low_summand(x):
        k = 1
        while True:
            if k == 1:
                next_term = -1/x - (1/2)*np.log(x)
                k += 1
            else:
                next_term = (sp.bernoulli(k)[-1]*(x**(k-1))/
                        (sp.factorial(k)*(k-1))
                )
                k += 2
            yield next_term
                
    def high_summand(x):
        k = 1
        while True:
            next_term = sp.expn(1,k*np.array(x,dtype='float64'))
            k += 1
            yield next_term
        
    err = 10*tol
    
    #Vectorized code starts here
    
    ind_both_low = np.intersect1d(np.where(a < bound), np.where(b < bound))
    ind_low_high = np.intersect1d(np.where(a < bound), np.where(b >= bound))
    ind_both_high = np.intersect1d(np.where(a >= bound), np.where(b >= bound))
    
    a_both_low = a[ind_both_low]
    b_both_low = b[ind_both_low]
    a_low_high = a[ind_low_high]
    b_low_high = b[ind_low_high]
    a_both_high = a[ind_both_high]
    b_both_high = b[ind_both_high]
    
    integral_both_low = np.array([])
    integral_low_high = np.array([])
    integral_both_high = np.array([])
    # Both Low
    if a_both_low.size > 0:
        low_sum_a = low_summand(a_both_low)
        low_sum_b = low_summand(b_both_low)
        integral_both_low = next(low_sum_b) - next(low_sum_a)

        while err > tol:
            next_term = next(low_sum_b) - next(low_sum_a)
            err = np.max(np.abs(next_term/integral_both_low))
            integral_both_low += next_term
    
    err = 10*tol
    
    # a Low b High
    if a_low_high.size > 0:
        low_sum_a = low_summand(a_low_high)
        high_sum_b = high_summand(b_low_high)
        low_sum_bound = low_summand(bound)
        int_bound_inf = np.float128(0.053082306482669888568)
        int_a_bound = next(low_sum_bound) - next(low_sum_a)
        int_bound_b = int_bound_inf - next(high_sum_b)
        integral_low_high = int_a_bound + int_bound_b

        while err > tol:
            next_term_a_bound = next(low_sum_bound) - next(low_sum_a)
            next_term_bound_b = - next(high_sum_b)
            next_term = next_term_a_bound + next_term_bound_b
            err = np.max(np.abs(next_term/integral_low_high))
            integral_low_high += next_term
    
    err = 10*tol
    
    # Both High
    if a_both_high.size > 0:
        high_sum_a = high_summand(a_both_high)
        high_sum_b = high_summand(b_both_high)
        integral_both_high = next(high_sum_a) - next(high_sum_b)

        while err > tol:
            next_term = next(high_sum_a) - next(high_sum_b)
            err = np.max(np.abs(next_term/integral_both_high))
            integral_both_high += next_term
    
    return np.concatenate((integral_both_low, integral_low_high, integral_both_high))
    
#     if a >= bound and b >= bound:
#         high_sum_a = high_summand(a)
#         high_sum_b = high_summand(b)
#         integral = next(high_sum_a) - next(high_sum_b)
    
#         while err > tol:
#             next_term = next(high_sum_a) - next(high_sum_b)
#             err = np.abs(next_term/integral)
#             integral += next_term
    
#     elif a < bound and b >= bound:
#         low_sum_a = low_summand(a)
#         high_sum_b = high_summand(b)
#         low_sum_bound = low_summand(bound)
#         int_bound_inf = 0.053082306482669888568
#         int_a_bound = next(low_sum_bound) - next(low_sum_a)
#         int_bound_b = int_bound_inf - next(high_sum_b)
#         integral = int_a_bound + int_bound_b
        
#         while err > tol:
#             next_term_a_bound = next(low_sum_bound) - next(low_sum_a)
#             next_term_bound_b = - next(high_sum_b)
#             next_term = next_term_a_bound + next_term_bound_b
#             err = np.abs(next_term/integral)
#             integral += next_term
    
#     else:
#         low_sum_a = low_summand(a)
#         low_sum_b = low_summand(b)
#         integral = next(low_sum_b) - next(low_sum_a)
        
#         while err > tol:
#             next_term = next(low_sum_b) - next(low_sum_a)
#             err = np.abs(next_term/integral)
#             integral += next_term
    
#     return integral
            

In [7]:
print(H(np.array([0.02124, 1e-6, 4.50204]),np.array([1.9033, 1e6, 9.10294]),tol=1e-12))

[  4.44616483971720413138e+01   9.99992461913929320872e+05
   2.06974344821578368908e-03]


In [8]:
def G(a, b, tol=1.0e-30):
    
    bound = 2
    
    def low_summand(x):
        k = 1
        while True:
            if k == 1:
                next_term = (1/2)*(np.log(x)**2) - (x/2)*(np.log(x) - 1)
                k += 1
            else:
                next_term = (sp.bernoulli(k)[-1]*(x**k)/
                        (sp.factorial(k)*(k**2))*(k*np.log(x) - 1)
                )
                k += 2
            yield next_term
                
    def high_summand(x):
        k = 1
        while True:
            next_term = (1/k)*(np.exp(-k*x)*np.log(x) + sp.expn(1,np.array(k*x,dtype=np.float64)))
            k += 1
            yield next_term
        
    err = 10*tol
    
    #Vectorized code starts here
    
    ind_both_low = np.intersect1d(np.where(a < bound), np.where(b < bound))
    ind_low_high = np.intersect1d(np.where(a < bound), np.where(b >= bound))
    ind_both_high = np.intersect1d(np.where(a >= bound), np.where(b >= bound))
    
    a_both_low = a[ind_both_low]
    b_both_low = b[ind_both_low]
    a_low_high = a[ind_low_high]
    b_low_high = b[ind_low_high]
    a_both_high = a[ind_both_high]
    b_both_high = b[ind_both_high]
    
    integral_both_low = np.array([])
    integral_low_high = np.array([])
    integral_both_high = np.array([])
    # Both Low
    if a_both_low.size > 0:
        low_sum_a = low_summand(a_both_low)
        low_sum_b = low_summand(b_both_low)
        integral_both_low = next(low_sum_b) - next(low_sum_a)

        while err > tol:
#             print('#######in G#######')
#             print(integral_both_low)
            next_term = next(low_sum_b) - next(low_sum_a)
            err = np.max(np.abs(next_term/integral_both_low))
            integral_both_low += next_term
    
    err = 10*tol
    
    # a Low b High
    if a_low_high.size > 0:
        low_sum_a = low_summand(a_low_high)
        high_sum_b = high_summand(b_low_high)
        low_sum_bound = low_summand(bound)
        int_bound_inf = np.float128(0.15171347859984083704)
        int_a_bound = next(low_sum_bound) - next(low_sum_a)
        int_bound_b = int_bound_inf - next(high_sum_b)
        integral_low_high = int_a_bound + int_bound_b

        while err > tol:
            next_term_a_bound = next(low_sum_bound) - next(low_sum_a)
            next_term_bound_b = - next(high_sum_b)
            next_term = next_term_a_bound + next_term_bound_b
            err = np.max(np.abs(next_term/integral_low_high))
            integral_low_high += next_term

    err = 10*tol
    
    # Both High
    if a_both_high.size > 0:
        high_sum_a = high_summand(a_both_high)
        high_sum_b = high_summand(b_both_high)
        integral_both_high = next(high_sum_a) - next(high_sum_b)

        while err > tol:
            next_term = next(high_sum_a) - next(high_sum_b)
            err = np.max(np.abs(next_term/integral_both_high))
            integral_both_high += next_term

    return np.concatenate((integral_both_low, integral_low_high, integral_both_high))
    
    
#     if a >= bound and b >= bound:
#         high_sum_a = high_summand(a)
#         high_sum_b = high_summand(b)
#         integral = next(high_sum_a) - next(high_sum_b)
    
#         while err > tol:
#             next_term = next(high_sum_a) - next(high_sum_b)
#             err = np.abs(next_term/integral)
#             integral += next_term
    
#     elif a < bound and b >= bound:
#         low_sum_a = low_summand(a)
#         high_sum_b = high_summand(b)
#         low_sum_bound = low_summand(bound)
#         int_bound_inf = 0.15171347859984083704
#         int_a_bound = next(low_sum_bound) - next(low_sum_a)
#         int_bound_b = int_bound_inf - next(high_sum_b)
#         integral = int_a_bound + int_bound_b
        
#         while err > tol:
#             next_term_a_bound = next(low_sum_bound) - next(low_sum_a)
#             next_term_bound_b = - next(high_sum_b)
#             next_term = next_term_a_bound + next_term_bound_b
#             err = np.abs(next_term/integral)
#             integral += next_term
    
#     else:
#         low_sum_a = low_summand(a)
#         low_sum_b = low_summand(b)
#         integral = next(low_sum_b) - next(low_sum_a)
        
#         while err > tol:
#             next_term = next(low_sum_b) - next(low_sum_a)
#             err = np.abs(next_term/integral)
#             integral += next_term
    
#     return integral
            

In [9]:
print(G(np.array([0.02124,1e-6,4.50204]),np.array([1.9033,1e6,9.10294]),tol=1e-20))

[ -6.90361462464218611501e+00  -9.47054794793618981430e+01
   1.85904820981936183544e-02]


# Check ICS Calculations

In [60]:
def F_nonrel(beta,photeng,T):
    eta = photeng/T
    if eta > 0.01:
        f2_at_0 = 4*eta**2*T**2/(np.exp(eta) - 1)**2*(
            np.exp(eta)*(eta - 1) + 1
        )

        f4_at_0 = 8*eta**2*T**2/(np.exp(eta) - 1)**4*(
            np.exp(2*eta)*(8*eta**3 + (4*eta**3 + 50*eta - 36)*np.cosh(eta)
                              + 2*(9 - 14*eta**2)*np.sinh(eta)
                            -50*eta + 27
                          )
            + 9
        )
        f6_at_0 = 4*eta**2*T**2/(np.exp(eta) - 1)**6*(
            np.exp(3*eta)*(
                -16*eta**2*np.sinh(eta)*(340*eta**2 + (68*eta**2 + 885)*np.cosh(eta) - 885)
                +3*(352*eta**5 - 3096*eta**3 + 6150*eta - 1125*np.sinh(eta) + 900*np.sinh(2*eta) - 2250)
                +np.cosh(eta)*(832*eta**5 + 6192*eta**3 - 24600*eta + 10125)
                +np.cosh(2*eta)*(32*eta**5 + 3096*eta**3 + 6150*eta - 4050)   
            )
            +675
        )
    else:
        f2_at_0 = T**2*(2*eta**2 + eta**5/45 - eta**7/1260 + eta**9/37800)
        f4_at_0 = T**2*(36*eta**2 - 68*eta**3/3 + 2*eta**5 - 89*eta**7/630 + 149*eta**9/18900)
        f6_at_0 = T**2*(1350*eta**2 - 1250*eta**3 + 1123*eta**5/5 - 2381*eta**7/84 + 6373*eta**9/2520)
    return (1+beta**2)/beta**2*2*(f2_at_0*beta**2/2 
                                  + f4_at_0*beta**4/24
                                 + f6_at_0*beta**6/720
                                 )

def H_nonrel(beta,photeng,T):
    eta = photeng/T
    if eta > 0.01:
        h1_at_0 = 2*eta**2*T**2/(np.exp(eta) - 1)
        h3_at_0 = 2*eta**2*T**2/(np.exp(eta) - 1)**3*(
            2*np.exp(eta)*(2*eta**2 + 9*eta - 15)
            + np.exp(2*eta)*(4*eta**2 - 18*eta + 15)
            +15
        )
    else:
        h1_at_0 = T**2*(2*eta - eta**2 + eta**3/6 - eta**5/360 + eta**7/15120 - eta**9/604800)
        h3_at_0 = T**2*(10*eta - 15*eta**2 - (31/120)*eta**5 + (37/3024)*eta**7 - (103/201600)*eta**9)
    return 2/beta*2*(h1_at_0*beta + h3_at_0*beta**3/6)

def K_nonrel(beta,photeng,T):
    eta = photeng/T
    if eta > 0.01:
        k2_at_0 = -4*eta**2*T**2/(np.exp(eta) - 1)**2*(
            np.exp(eta)*(eta - 1) + 1
        )
        k4_at_0 = 8*eta**2*T**2/(np.exp(eta) - 1)**4*(
            np.exp(2*eta)*(-8*eta**3 
                           - 2*(2*eta**3 + 7*eta + 2)*np.cosh(eta)
                           +(20*eta**2 + 2)*np.sinh(eta)
                           +14*eta + 3
                          )
            +1
        )
    else:
        k2_at_0 = T**2*(-2*eta**2 + 2*eta**3/3 - eta**5/45 + eta**7/1260 - eta**9/37800)
        k4_at_0 = T**2*(4*eta**2 + 4*eta**3 - 46*eta**5/45 + 59*eta**7/630 - 37*eta**9/6300)
    return 1/beta**2*2*(k2_at_0*beta**2/2 
                            + k4_at_0*beta**4/24
                        )

def G_nonrel(beta,photeng,T):
    eta = photeng/T
    if eta > 0.01:
        g2_at_0 = -4*eta**2*T**2/(np.exp(eta) - 1)
        g4_at_0 = -16*eta**2*T**2/(np.exp(eta) - 1)**3*(
            np.exp(2*eta)*(eta**2-3*eta+3)
            + np.exp(eta)*(eta**2+3*eta-6)
            +3
        )
    else:
        g2_at_0 = T**2*(-4*eta + 2*eta**2 - eta**3/3 + eta**5/180 - eta**7/7560 + eta**9/302400)
        g4_at_0 = T**2*(-32*eta + 24*eta**2 - 8*eta**3 + 2*eta**5/5 - 19*eta**7/945 + 11*eta**9/12600)
    gamma = np.sqrt(1/(1-beta**2))
    print('**G_nonrel check**')
    print(2*beta**2*np.sqrt(1-beta)/(gamma*beta**2)*2*(g2_at_0*beta**2/2))
    print(2*beta**2*np.sqrt(1-beta)/(gamma*beta**2)*2*(g4_at_0*beta**4/24))
    print('**end G_nonrel check**')
    return 2/(gamma*beta**2)*2*(g2_at_0*beta**2/2 + g4_at_0*beta**4/24)

def diff1(beta,photeng,T):
    eta = photeng/T
    if eta > 0.01:
        f2_at_0 = 4*eta**2*T**2/(np.exp(eta) - 1)**2*(
            np.exp(eta)*(eta - 1) + 1
        )
        f4_at_0 = 8*eta**2*T**2/(np.exp(eta) - 1)**4*(
            np.exp(2*eta)*(8*eta**3 + (4*eta**3 + 50*eta - 36)*np.cosh(eta)
                              + 2*(9 - 14*eta**2)*np.sinh(eta)
                            -50*eta + 27
                          )
            + 9
        )
        k4_at_0 = 8*eta**2*T**2/(np.exp(eta) - 1)**4*(
            np.exp(2*eta)*(-8*eta**3 
                           - 2*(2*eta**3 + 7*eta + 2)*np.cosh(eta)
                           +(20*eta**2 + 2)*np.sinh(eta)
                           +14*eta + 3
                          )
            +1
        )
    else:
        f2_at_0 = T**2*(2*eta**2 + eta**5/45 - eta**7/1260 + eta**9/37800)
        f4_at_0 = T**2*(36*eta**2 - 68*eta**3/3 + 2*eta**5 - 89*eta**7/630 + 149*eta**9/18900)
        k4_at_0 = T**2*(4*eta**2 + 4*eta**3 - 46*eta**5/45 + 59*eta**7/630 - 37*eta**9/6300)
    
    print('**diff1 test**')
    print(2*f2_at_0*beta**2/2)
    print((1/12)*beta**2*(f4_at_0 + k4_at_0))
    print('**end diff1 test**')
    return 2*f2_at_0*beta**2/2 + (1/12)*beta**2*(f4_at_0 + k4_at_0)
    
def diff2(beta,photeng,T):
    eta = photeng/T
    if eta > 0.01:
        g2_at_0 = -4*eta**2*T**2/(np.exp(eta) - 1)
        g4_at_0 = -16*eta**2*T**2/(np.exp(eta) - 1)**3*(
            np.exp(2*eta)*(eta**2-3*eta+3)
            + np.exp(eta)*(eta**2+3*eta-6)
            +3
        )
        h3_at_0 = 2*eta**2*T**2/(np.exp(eta) - 1)**3*(
            2*np.exp(eta)*(2*eta**2 + 9*eta - 15)
            + np.exp(2*eta)*(4*eta**2 - 18*eta + 15)
            +15
        )
    else:
        g2_at_0 = T**2*(-4*eta + 2*eta**2 - eta**3/3 + eta**5/180 - eta**7/7560 + eta**9/302400)
        g4_at_0 = T**2*(-32*eta + 24*eta**2 - 8*eta**3 + 2*eta**5/5 - 19*eta**7/945 + 11*eta**9/12600)
        h3_at_0 = T**2*(10*eta - 15*eta**2 - (31/120)*eta**5 + (37/3024)*eta**7 - (103/201600)*eta**9)
    print('**start diff2 test**')
    print(4*beta**2*h3_at_0/6)
    print(4*beta**2*np.sqrt(1-beta**2)*g4_at_0/24)
    print(beta**2*g2_at_0)
    print('**end diff2 test**')
    return 4*beta**2*h3_at_0/6 + 4*beta**2*np.sqrt(1-beta**2)*g4_at_0/24 - beta**2*g2_at_0

In [71]:
eleceng = np.array([51099800.94610007945],dtype=np.float128)
photeng = np.array([7.9432e-8],dtype=np.float128)
T = np.float128(phys.TCMB(1)*1000)

gamma = np.float128(eleceng/phys.me)
beta = np.sqrt(1 - 1/gamma**2)

print(eleceng-phys.me)
print(beta)
print(photeng/T)
print(np.exp(photeng/T) - 1)

prefac = np.float128(phys.c*(3/8)*phys.thomson_xsec/(2*gamma**3*beta**2)
          *(8*np.pi/(phys.ele_compton*phys.me)**3)
         )

minratio = (1-beta)/(1+beta)*photeng/T
maxratio = (1+beta)/(1-beta)*photeng/T

minratiotest = minratio[0]
maxratiotest = maxratio[0]
photengtest = photeng[0]/T

quadxlow  = quad(lambda x: x/(np.exp(x) - 1), minratiotest, photengtest, epsrel=1e-20)[0]*T**2
quadxhigh = quad(lambda x: x/(np.exp(x) - 1), photengtest, maxratiotest, epsrel=1e-20)[0]*T**2

quad1low  = quad(lambda x: 1/(np.exp(x) - 1), minratiotest, photengtest, epsrel=1e-20)[0]*T
quad1high = quad(lambda x: 1/(np.exp(x) - 1), photengtest, maxratiotest, epsrel=1e-20)[0]*T

quadinvlow  = quad(lambda x: (1/x)/(np.exp(x) - 1), minratiotest, photengtest, epsrel=1e-20)[0]
quadinvhigh = quad(lambda x: (1/x)/(np.exp(x) - 1), photengtest, maxratiotest, epsrel=1e-20)[0]

quadloglow  = quad(lambda x: np.log(x)/(np.exp(x) - 1), minratiotest, photengtest, epsrel=1e-20)[0]*T
quadloghigh = quad(lambda x: np.log(x)/(np.exp(x) - 1), photengtest, maxratiotest, epsrel=1e-20)[0]*T 


# quadfac1 = (1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*quadxlow
# quadfac2 = (2/beta)*np.sqrt((1-beta)/(1+beta))*photeng*quad1low
# quadfac3 = -(1-beta)**2/(beta**2)*np.sqrt((1+beta)/(1-beta))*photeng**2*quadinvlow
# quadfac4 = 2/(gamma*beta**2)*photeng*np.log((1-beta)/(1+beta)*photeng)*quad1low
# quadfac5 = -2/(gamma*beta**2)*photeng*(quadloglow + np.log(T)*quad1low)

# quadfac6 = -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*quadxhigh
# quadfac7 = (2/beta)*np.sqrt((1+beta)/(1-beta))*photeng*quad1high
# quadfac8 = (1+beta)/(gamma*beta**2)*photeng**2*quadinvhigh
# quadfac9 = -2/(gamma*beta**2)*photeng*np.log((1+beta)/(1-beta)*photeng)*quad1high
# quadfac10 = 2/(gamma*beta**2)*photeng*(quadloghigh + np.log(T)*quad1high)

quadfac1 = np.array([quadxlow],dtype=np.float128)
quadfac2 = 2*beta/(1+beta**2)*(1-beta)/(1+beta)*photeng*quad1low
quadfac3 = -(1-beta)**2/(1+beta**2)*photeng**2*quadinvlow
quadfac4 = 2*(1-beta)/(1+beta**2)*photeng*np.log((1-beta)/(1+beta)*photeng)*quad1low
quadfac5 = -2*(1-beta)/(1+beta**2)*photeng*(quadloglow + np.log(T)*quad1low)

quadfac6 = np.array([-quadxhigh],dtype=np.float128)
quadfac7 = 2*beta/(1+beta**2)*(1+beta)/(1-beta)*photeng*quad1high
quadfac8 = (1+beta)**2/(1+beta**2)*photeng**2*quadinvhigh
quadfac9 = -2*(1+beta)/(1+beta**2)*photeng*np.log((1+beta)/(1-beta)*photeng)*quad1high
quadfac10 = 2*(1+beta)/(1+beta**2)*photeng*(quadloghigh + np.log(T)*quad1high)

quadpartlow = quadfac1 + quadfac2 + quadfac3 + quadfac4 + quadfac5
quadparthigh = quadfac6 + quadfac7 + quadfac8 + quadfac9 + quadfac10

print('****Parameters****')
print(minratio)
print(photeng/T)
print(maxratio)

# print('***Computed using Quadrature***')
# print('First Term:')
# print(prefac*quadfac1)
# print(prefac*quadfac6)
# print('Second Term:')
# print(prefac*quadfac2)
# print(prefac*quadfac7)
# print('Third Term:')
# print(prefac*quadfac3)
# print(prefac*quadfac8)
# print('Fourth Term:')
# print(prefac*quadfac4)
# print(prefac*quadfac9)
# print('Fifth Term:')
# print(prefac*quadfac5)
# print(prefac*quadfac10)
# print('Actual:')
# print(prefac*(quadpartlow + quadparthigh))


# fac1 = (1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*F1(minratio,photeng/T)*T**2
# fac2 = (2/beta)*np.sqrt((1-beta)/(1+beta))*photeng*F0(minratio,photeng/T)*T
# fac3 = -(1-beta)**2/(beta**2)*np.sqrt((1+beta)/(1-beta))*photeng**2*H(minratio,photeng/T)
# fac4 = 2/(gamma*beta**2)*photeng*np.log((1-beta)/(1+beta)*photeng)*F0(minratio,photeng/T)*T
# fac5 = -2/(gamma*beta**2)*photeng*(G(minratio,photeng/T)*T + T*np.log(T)*F0(minratio,photeng/T))

# fac6 = -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*F1(photeng/T,maxratio)*T**2
# fac7 = (2/beta)*np.sqrt((1+beta)/(1-beta))*photeng*F0(photeng/T,maxratio)*T
# fac8 = (1+beta)/(gamma*beta**2)*photeng**2*H(photeng/T,maxratio)
# fac9 = -2/(gamma*beta**2)*photeng*np.log((1+beta)/(1-beta)*photeng)*F0(photeng/T,maxratio)*T
# fac10 = 2/(gamma*beta**2)*photeng*(G(photeng/T,maxratio)*T + T*np.log(T)*F0(photeng/T,maxratio))


fac1 = F1(minratio,photeng/T)*T**2
fac2 = 2*beta/(1+beta**2)*(1-beta)/(1+beta)*photeng*F0(minratio,photeng/T)*T
fac3 = -(1-beta)**2/(1+beta**2)*photeng**2*H(minratio,photeng/T)
fac4 = 2*(1-beta)/(1+beta**2)*photeng*np.log((1-beta)/(1+beta)*photeng)*F0(minratio,photeng/T)*T
fac5 = -2*(1-beta)/(1+beta**2)*photeng*(G(minratio,photeng/T)*T + T*np.log(T)*F0(minratio,photeng/T))

fac6 = -F1(photeng/T,maxratio)*T**2
fac7 = 2*beta/(1+beta**2)*(1+beta)/(1-beta)*photeng*F0(photeng/T,maxratio)*T
fac8 = (1+beta)**2/(1+beta**2)*photeng**2*H(photeng/T,maxratio)
fac9 = -2*(1+beta)/(1+beta**2)*photeng*np.log((1+beta)/(1-beta)*photeng)*F0(photeng/T,maxratio)*T
fac10 = 2*(1+beta)/(1+beta**2)*photeng*(G(photeng/T,maxratio)*T + T*np.log(T)*F0(photeng/T,maxratio))



part_low = fac1+fac2+fac3+fac4+fac5
part_high = fac6+fac7+fac8+fac9+fac10
# print('***Computed using Series***')
# print('First Term:')
# print(prefac*fac1)
# print(prefac*fac6)
# print('Second Term:')
# print(prefac*fac2)
# print(prefac*fac7)
# print('Third Term:')
# print(prefac*fac3)
# print(prefac*fac8)
# print('Fourth Term:')
# print(prefac*fac4)
# print(prefac*fac9)
# print('Fifth Term:')
# print(prefac*fac5)
# print(prefac*fac10)
# print('Actual:')
# print(prefac*part_low + prefac*part_high)

print('***Comparison (Series followed by Quadrature)***')
print('1st Term:')
print(fac1)
print(quadfac1)
print('2nd Term:')
print(fac2)
print(quadfac2)
print('3rd Term:')
print(fac3)
print(quadfac3)
print('4th Term:')
print(fac4)
print(quadfac4)
print('5th Term:')
print(fac5)
print(quadfac5)
print('6th Term:')
print(fac6)
print(quadfac6)
print('7th Term:')
print(fac7)
print(quadfac7)
print('8th Term:')
print(fac8)
print(quadfac8)
print('9th Term:')
print(fac9)
print(quadfac9)
print('10th Term:')
print(fac10)
print(quadfac10)
print('Actual:')
test1 = (1+beta**2)*np.sqrt(1+beta)*fac1 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*fac6
test2 = (1+beta**2)*np.sqrt(1+beta)*fac2 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*fac7
test3 = (1+beta**2)*np.sqrt(1+beta)*fac3 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*fac8
test4 = (1+beta**2)*np.sqrt(1+beta)*fac4 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*fac9
test5 = (1+beta**2)*np.sqrt(1+beta)*fac5 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*fac10
testquad1 = (1+beta**2)*np.sqrt(1+beta)*quadfac1 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*quadfac6
testquad2 = (1+beta**2)*np.sqrt(1+beta)*quadfac2 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*quadfac7
testquad3 = (1+beta**2)*np.sqrt(1+beta)*quadfac3 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*quadfac8
testquad4 = (1+beta**2)*np.sqrt(1+beta)*quadfac4 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*quadfac9
testquad5 = (1+beta**2)*np.sqrt(1+beta)*quadfac5 + (1+beta**2)*(1-beta)/np.sqrt(1+beta)*quadfac10

testdiff1=beta**2*np.sqrt(1-beta)*F_nonrel(beta,photeng,T)
testdiff2=beta**2*np.sqrt(1-beta)*H_nonrel(beta,photeng,T)
testdiff3=beta**2*np.sqrt(1-beta)*K_nonrel(beta,photeng,T)
testdiff4=beta**2*np.sqrt(1-beta)*G_nonrel(beta,photeng,T)
print('1 and 6 Term Difference')
print(test1)
print(testquad1)
print(testdiff1)
print('2 and 7 Term Difference')
print(test2)
print(testquad2)
print(testdiff2)
print('3 and 8 Term Difference')
print(test3)
print(testquad3)
print(testdiff3)
print('4 and 9 Term Difference')
print(test4)
print(testquad4)
print('5 and 10 Term Difference')
print(test5)
print(testquad5)
print('4+5 and 9+10 Term Difference')
print(test4+test5)
print(testquad4+testquad5)
print(testdiff4)
print('Sum...')
print(test1 + test2 + test3 + test4 + test5)
print(testquad1 + testquad2 + testquad3 + testquad4 + testquad5)
print(testdiff1+testdiff2+testdiff3+testdiff4)
print((diff1(beta,photeng,T) + diff2(beta,photeng,T))*beta**2*np.sqrt(1-beta))

print('***Errors in each term (quad - series)/quad***')
print((quadfac1 - fac1)/quadfac1)
print((quadfac2 - fac2)/quadfac2)
print((quadfac3 - fac3)/quadfac3)
print((quadfac4 - fac4)/quadfac4)
print((quadfac5 - fac5)/quadfac5)
print((quadfac6 - fac6)/quadfac6)
print((quadfac7 - fac7)/quadfac7)
print((quadfac8 - fac8)/quadfac8)
print((quadfac9 - fac9)/quadfac9)
print((quadfac10 - fac10)/quadfac10)


[ 50588802.000000078522]
[ 0.99994999856663214539]
[ 3.3800851063829791753e-07]
[ 3.3800856776315321706e-07]
****Parameters****
[ 8.4506662839378693638e-12]
[ 3.3800851063829791753e-07]
[ 0.013519614835705224347]
***Comparison (Series followed by Quadrature)***
1st Term:
[ 1.8666051734593875006e-08]
[ 1.8666051737797236737e-08]
2nd Term:
[ 4.9452976215337639134e-12]
[ 4.9452990552535187077e-12]
3rd Term:
[-9.3335275478111816025e-13]
[-9.3335253070731395438e-13]
4th Term:
[-2.6650821376319732774e-10]
[-2.665082910281306299e-10]
5th Term:
[ 2.1410365711929383149e-10]
[ 2.1410373797923341013e-10]
6th Term:
[-0.00074408234736337988682]
[-0.00074408234736338706119]
7th Term:
[ 0.0079065858730297109251]
[ 0.007906585873073634442]
8th Term:
[ 3.7332039756270171948e-08]
[ 3.733203975620608836e-08]
9th Term:
[ 2.2740273831614991459e-06]
[ 2.2740273831741320674e-06]
10th Term:
[-4.3698450594931513287e-06]
[-4.3698450595281107388e-06]
Actual:
**G_nonrel check**
[-1.0558491264168255474e-11]
[-7.03

/Users/hongwan/anaconda/envs/python3/lib/python3.6/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


In [12]:
print(minratio)
print(photeng/T)
print(T)
F1(minratio,photeng/T)*T**2

[ 3.3800813365791140106e-07]
[ 3.3800851063829791753e-07]
0.235


array([ 2.0818738326917676029e-14], dtype=float128)

In [13]:
quad_result1 = quad(lambda x: np.log(x)/(np.exp(x) - 1), minratiotest, photengtest, epsrel=1e-13)
quad_result2 = quad(lambda x: 1/(np.exp(x) - 1),
           minratiotest, photengtest, epsrel=1e-13)
print((2*prefac/(gamma*beta**2))*photeng*(T*quad_result1[0] + T*np.log(T)*quad_result2[0]))
print((2*prefac/(gamma*beta**2))*photeng*(T*G(minratio,photeng/T) + T*np.log(T)*F0(minratio,photeng/T)))
print(minratio, photeng/T)

[-347103126762.6121285]
[-347103126772.96458817]
[ 3.3800813365791140106e-07] [ 3.3800851063829791753e-07]


In [14]:
eleceng = 510998.966053
photeng = 7.94328e-8
T = phys.TCMB(1)
gamma = eleceng/phys.me
beta = np.sqrt(1 - 1/gamma**2)
lowlim = photeng*(1-beta)/(1+beta)
upplim = photeng*(1+beta)/(1-beta)
def test_func_int(x):
    ratio = photeng/x
    prefac = (phys.c*(3/16)*phys.thomson_xsec*ratio/(x*gamma**3*beta**2)
          *(8*np.pi/(phys.ele_compton*phys.me)**3)
         )
    if ratio < 1: 
        beta_fac = (1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))/ratio
    else:
        beta_fac = -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))/ratio
    return beta_fac*prefac*x**2/(np.exp(x/T) - 1)

quad(test_func_int, lowlim, upplim)

(4.713112431502242e-05, 4.6629367034256575e-14)

In [15]:
eleceng = 510998.966053
photeng = 7.94328e-8
T = phys.TCMB(1)
gamma = eleceng/phys.me
beta = np.sqrt(1 - 1/gamma**2)
lowlim = photeng*(1-beta)/(1+beta)/T
upplim = photeng*(1+beta)/(1-beta)/T
prefac = (phys.c*(3/16)*phys.thomson_xsec/(gamma**3*beta**2)
          *(8*np.pi/(phys.ele_compton*phys.me)**3)*T**2
         )

def test_func_int(x):
    if x < photeng/T: 
        beta_fac = (1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))
    else:
        beta_fac = -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))
    return beta_fac*prefac*x/(np.exp(x) - 1)

a = (1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*F1(lowlim,photeng/T)
b = -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*F1(photeng/T,upplim)
print(prefac*(a+b))
print(quad(test_func_int, lowlim, upplim))
print(quad(test_func_int, lowlim, photeng/T))
print(quad(test_func_int, photeng/T, upplim))

print(prefac*((1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*F1(lowlim,photeng/T)))
print(prefac*(-(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*F1(photeng/T,upplim)))
print(prefac*((1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*F1(lowlim,photeng/T) 
     -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*F1(photeng/T,upplim)))

print(prefac*((1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*quad(lambda x: x/(np.exp(x) - 1), lowlim, photeng/T)[0]))
print(prefac*(-(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*quad(lambda x: x/(np.exp(x) - 1), photeng/T, upplim)[0]))
print(prefac*((1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*quad(lambda x: x/(np.exp(x) - 1), lowlim, photeng/T)[0]
      -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*quad(lambda x: x/(np.exp(x) - 1), photeng/T, upplim)[0]))

7.96723065208e-09
(4.714705882946646e-05, 4.657385588302532e-14)
(0.08434165427102648, 1.5482116679935564e-14)
(-0.08434164630377111, 2.1084352112281497e-14)
0.084341654271
-0.0843416463038
7.96723065208e-09
0.084341654271
-0.0843416463038
7.9672553364e-09


In [16]:
print(test_func_int(photeng/T*0.99999999))
print(test_func_int(photeng/T*1.00000001))

446572.887617
-446323.364902


In [17]:
print(phys.c)
print(phys.thomson_xsec)
print(phys.me)
print(np.pi)
print(gamma)
print(beta)

29979245800.0
6.652458734e-25
510998.9461
3.141592653589793
1.0000000390470474
0.000279453198576


In [18]:
minratiotest = 3.378e-7
photengtest = 3.380e-7
maxratiotest = 3.382e-7
testquad1 = quad(lambda x: x/(np.exp(x) - 1), minratiotest, photengtest, epsrel=1e-18)
testquad2 = quad(lambda x: x/(np.exp(x) - 1), photengtest, maxratiotest, epsrel=1e-18)
testsum1 = F1(minratiotest,photengtest)
testsum2 = F1(photengtest,maxratiotest)
print(testquad1)
print(testsum1)
print(testquad2)
print(testsum2)

(1.9999996619753332e-10, 3.229867589950235e-20)
1.9999996620996494e-10
(1.999999662001018e-10, 2.971854187384124e-20)
1.9999996619000672e-10


In [19]:
def icsspec(eleceng_arr, photeng_arr, T):
    
    gamma_arr = eleceng_arr/phys.me
    beta_arr = np.sqrt(1 - 1/gamma_arr**2)
    
    prefac = np.array([phys.c*(3/8)*phys.thomson_xsec/(2*gamma**3*beta**2)
          *(8*np.pi/(phys.ele_compton*phys.me)**3)*np.ones(photeng_arr.size)
        for beta,gamma in zip(beta_arr,gamma_arr)])
    
    minratio_arr = np.array([(1-beta)/(1+beta)*photeng_arr/T for beta in beta_arr])
    maxratio_arr = np.array([(1+beta)/(1-beta)*photeng_arr/T for beta in beta_arr])
    
#     utils.compare_arr([minratio_arr[0], photeng_arr/T, maxratio_arr[0]])
    
    fac = np.array([(1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*F1(minratio,photeng_arr/T)*T**2
                     -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*F1(photeng_arr/T,maxratio)*T**2
                     +(2/beta)*np.sqrt((1-beta)/(1+beta))*photeng_arr*F0(minratio,photeng_arr/T)*T
                     +(2/beta)*np.sqrt((1+beta)/(1-beta))*photeng_arr*F0(photeng_arr/T,maxratio)*T
                     -(1-beta)**2/(beta**2)*np.sqrt((1+beta)/(1-beta))*photeng_arr**2*H(minratio,photeng_arr/T)
                     +(1+beta)/(gamma*beta**2)*photeng_arr**2*H(photeng_arr/T,maxratio)
                     +2/(gamma*beta**2)*photeng_arr*np.log((1-beta)/(1+beta)*photeng_arr)*F0(minratio,photeng_arr/T)*T
                     -2/(gamma*beta**2)*photeng_arr*np.log((1+beta)/(1-beta)*photeng_arr)*F0(photeng_arr/T,maxratio)*T
                     -2/(gamma*beta**2)*photeng_arr*(G(minratio,photeng_arr/T)*T + T*np.log(T)*F0(minratio,photeng_arr/T))
                     +2/(gamma*beta**2)*photeng_arr*(G(photeng_arr/T,maxratio)*T + T*np.log(T)*F0(photeng_arr/T,maxratio))
                    for (gamma, beta, minratio, maxratio) in zip(gamma_arr, beta_arr, minratio_arr, maxratio_arr)
                    ])
    
    print(fac)
    
    return prefac*fac


In [20]:
Emax = 1e10
Emin = 1e-8
nEe = 10
nEp  = 10

dlnEp = np.log(Emax/Emin)/nEp
lowengEp = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

dlnEe = np.log(Emax/Emin)/nEe
lowengEe = phys.me + Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

In [21]:
lowengEp[0]
lowengEe[0]

510998.94610007945

In [22]:
%%time
spec = icsspec(lowengEe, lowengEp, phys.TCMB(1)*1000)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [23]:
print(spec[3,0])
print(lowengEe[3])
print(lowengEp[0])

NameError: name 'spec' is not defined

In [ ]:
# testeleceng = np.array([lowengEe[0]])
testeleceng = np.array([510998.966053])
testphoteng = np.array([7.94328e-8])
T = phys.TCMB(1)*1000
print(icsspec(testeleceng, testphoteng, T))

In [ ]:
print(lowengEe[0] - 510998.9461)

In [ ]:
def test_piecewise(x):
    if x > 0:
        return -4000000
    else:
        return 4000000

print(quad(test_piecewise, -1, 1))
print(quad(test_piecewise, -1, 0)[0]+quad(test_piecewise, 0, 1)[0])
print(quad(test_piecewise, -1, 0)[0])
print(quad(test_piecewise, 0, 1)[0])